[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [4]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df=pd.read_csv('labeledTrainData.csv', sep='\t',encoding='latin-1')
df.head()


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [6]:
df.shape

(25000, 3)

## 2. Preprocessing

In [62]:
# stop words
from nltk.corpus import stopwords
stop=stopwords.words('english')




In [63]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>','',text)
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

a='Intrigued by the synopsis (every gay video these days has a hunk on the \
cover; this is not necessarily to be construed as a good sign) \
I purchased BEN AND ARTHUR without knowing a thing about it'

print(preprocessor(a))
print(a)

intrigued by the synopsis every gay video these days has a hunk on the cover this is not necessarily to be construed as a good sign i purchased ben and arthur without knowing a thing about it 
Intrigued by the synopsis (every gay video these days has a hunk on the cover; this is not necessarily to be construed as a good sign) I purchased BEN AND ARTHUR without knowing a thing about it


In [65]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter=PorterStemmer()


def tokenizer_porter(text):
    return [porter.stem(x) for x in text.split()]

In [101]:
# split the dataset in train and test
# Your code here

from sklearn.model_selection import train_test_split

X=df['review']
y=df['sentiment']

X_train, X_test, y_train,  y_test=train_test_split(X,y,test_size=0.3, random_state=101, stratify=df['sentiment'])



In [102]:
y_test.value_counts()

1    3750
0    3750
Name: sentiment, dtype: int64

In [103]:
y_train.shape

(17500,)

In [104]:
y_train.value_counts()

1    8750
0    8750
Name: sentiment, dtype: int64

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [105]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)# Your code here





In [106]:
clf=Pipeline([('vect',tfidf),
             ('model',LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

C:\Users\Khanh Nguyen\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\Khanh Nguyen\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [111]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score ,classification_report

predictions=clf.predict(X_test)
print('Accuracy score: ', accuracy_score(y_test, predictions))
print('Confusion matrix:\n ', confusion_matrix(y_test, predictions))
print('classification_report:\n\n ', classification_report(y_test, predictions))

Accuracy score:  0.8856
Confusion matrix:
  [[3249  501]
 [ 357 3393]]
classification_report:

                precision    recall  f1-score   support

           0       0.90      0.87      0.88      3750
           1       0.87      0.90      0.89      3750

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



In [121]:
twits = [
    "love love love",
    "I love this!",
    "I would love to have that two hours of my life back. It seemed to be several clips from Steve's Animal\
    Planet series that was spliced into a loosely constructed script. Don't Go"
]
prediction=clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]}->> Nagative, positive :{prediction[i]}')
          

love love love->> Nagative, positive :[0.00844076 0.99155924]
I love this!->> Nagative, positive :[0.0212408 0.9787592]
I would love to have that two hours of my life back. It seemed to be several clips from Steve's Animal    Planet series that was spliced into a loosely constructed script. Don't Go->> Nagative, positive :[0.47979223 0.52020777]


## 5. Export Model 

In [119]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('./','logisticRegression.pkl'),'wb'), protocol=4)